In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
#컬럼 한꺼번에보기
from IPython.display import display
pd.options.display.max_columns = None
import platform
platform.system()
import pymysql

# 운영체제별 한글 폰트 설정
if platform.system() == 'Darwin': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # Windows 환경 폰트 설정
    plt.rc('font', family='Malgun Gothic')

plt.rc('axes', unicode_minus=False) # 마이너스 폰트 설정

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
print("pandas version: ", pd.__version__)
pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 100)

# 글씨 선명하게 출력하는 설정
%config InlineBackend.figure_format = 'retina'

pandas version:  1.1.5


In [ ]:
import pymysql

conn = pymysql.connect(host='127.0.0.1',
                      user='root',
                      password='maria',
                      database='aps',
                      port=3306,
                      charset='utf8',
                      autocommit=True,
                      cursorclass=pymysql.cursors.DictCursor
                      )
cursor = conn.cursor()

sql = " SELECT orderid, solddate, custid, prodname, \
                (CASE DAYOFWEEK(solddate) \
                      WHEN '1' THEN '일' \
                      WHEN '2' THEN '월' \
                      WHEN '3' THEN '화' \
                      WHEN '4' THEN '수' \
                      WHEN '5' THEN '목' \
                      WHEN '6' THEN '금' \
                      WHEN '7' THEN '토' \
                      END) AS weekday, \
                sitebusname, region, \
                d_day_2_tem, d_day_2_hum, d_day_2_wind, d_day_2_tem_rainamount, d_day_2_rainhour, order_quant, \
                IF (solddate > '2021-03-31', 0, sold_quant) sold_quant \
         FROM sales_new \
         WHERE solddate < '2021-04-04' \
         UNION ALL \
         SELECT a.orderid, a.solddate, a.custid, a.prodname,  \
                (CASE DAYOFWEEK(a.solddate) \
                      WHEN '1' THEN '일' \
                      WHEN '2' THEN '월' \
                      WHEN '3' THEN '화' \
                      WHEN '4' THEN '수' \
                      WHEN '5' THEN '목' \
                      WHEN '6' THEN '금' \
                      WHEN '7' THEN '토' \
                      END) AS weekday, \
                a.sitebusname, a.region, \
                b.d_day_2_tem, b.d_day_2_hum, b.d_day_2_wind, b.d_day_2_tem_rainamount, b.d_day_2_rainhour, \
                a.order_quant, 0 sold_quant  \
        FROM pred_order a, weather b, area c \
        WHERE a.solddate = b.solddate \
        AND c.area_stat = b.region \
        AND a.region = c.area_name \
        AND a.order_quant > 0 "

cursor.execute(sql)
row = cursor.fetchall() 
conn.close()
print(row)

In [ ]:
df = pd.DataFrame(row)
print(df.head())

In [27]:
df = df.reset_index().rename(columns={"index": "index"})
df

,index,orderid,solddate,custid,prodname,weekday,sitebusname,region,d_day_2_tem,d_day_2_hum,d_day_2_wind,d_day_2_tem_rainamount,d_day_2_rainhour,order_quant,sold_quant
0,0,AE21030434,2021-03-07,2007400,AE,일,삼표,풍납,6.6,58.5,2.2,0.0,0.0,532,1000
1,1,HR1021030423,2021-03-07,2001500,PEMA-HR1000,일,제일,홍성,5.5,65.0,0.9,0.0,0.0,5184,6000
2,2,HR1021030426,2021-03-07,2001600,PEMA-HR1000,일,동해,동해,8.1,62.1,2.1,0.0,0.0,10032,8000
3,3,HR1021030427,2021-03-07,2001700,PEMA-HR1000,일,동해,동해,8.1,62.1,2.1,0.0,0.0,1344,2000
4,4,HR1521030432,2021-03-07,2002800,PEMA-HR1500,일,신일,송파,6.6,58.5,2.2,0.0,0.0,6260,5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28576,28576,20211009_949,2021-10-09,2007400,CSA5000,토,승일,함안,18.6,55.4,3.4,0.0,0.0,6396,0
28577,28577,20211009_3062,2021-10-09,2007500,PEMA-HR1000,토,삼우,경산,17.8,59.1,3.7,0.0,0.0,7602,0
28578,28578,20211009_3063,2021-10-09,2007600,PEMA-HR1000,토,한빛,부안,17.2,64.6,1.7,0.0,0.0,5836,0
28579,28579,20211009_3064,2021-10-09,2007700,PEMA-HR1000,토,현진,동해,16.6,63.9,3.3,0.0,0.0,8141,0


In [28]:
# 종속변수(sold_quant),독립변수(ORDER_QUANT) 로그화(편향완화 확인)

df['log_SOLD_QUANT'] = np.log1p(df['sold_quant'].astype('float'))
df['log_ORDER_QUANT'] = np.log1p(df['order_quant'].astype('float'))

In [29]:
# 최종 DB병합용 데이터프레임 생성

# 2021년 4월~10월 데이터 추출
df_pred_21_03 = df[df['solddate'].str.contains('2021-04-|2021-05-|\
2021-06-|2021-07-|2021-08-|2021-09-|2021-10-')]

# 2021년 4월~10월  데이터이외
df_pred_21_03x = df[~df.solddate.str.contains('2021-04-|2021-05-|\
2021-06-|2021-07-|2021-08-|2021-09-|2021-10-')]

In [30]:
# 병합용 2018.01~2021.03 데이터셋 분리
df_pred_21_03 = df_pred_21_03.drop(['sold_quant','order_quant'],axis=1)
df_pred_21_03

,index,orderid,solddate,custid,prodname,weekday,sitebusname,region,d_day_2_tem,d_day_2_hum,d_day_2_wind,d_day_2_tem_rainamount,d_day_2_rainhour,log_SOLD_QUANT,log_ORDER_QUANT
931,931,AE21032954,2021-04-01,2006200,AE,목,신일,송파,14.9,80.8,3.1,56.2,16.5,0.0,7.412764
932,932,AE21032953,2021-04-01,2006100,AE,목,삼표,용인,15.4,85.4,1.9,46.7,14.6,0.0,6.532334
933,933,AE21032952,2021-04-01,2006000,AE,목,삼표,풍납,14.9,80.8,3.1,56.2,16.5,0.0,6.897705
934,934,AE21032951,2021-04-01,2005900,AE,목,삼표,성수,14.9,80.8,3.1,56.2,16.5,0.0,7.873598
935,935,HR1021032950,2021-04-01,2001600,PEMA-HR1000,목,성진,인천,14.1,79.6,3.1,54.7,16.5,0.0,9.021719
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28576,28576,20211009_949,2021-10-09,2007400,CSA5000,토,승일,함안,18.6,55.4,3.4,0.0,0.0,0.0,8.763584
28577,28577,20211009_3062,2021-10-09,2007500,PEMA-HR1000,토,삼우,경산,17.8,59.1,3.7,0.0,0.0,0.0,8.936298
28578,28578,20211009_3063,2021-10-09,2007600,PEMA-HR1000,토,한빛,부안,17.2,64.6,1.7,0.0,0.0,0.0,8.671972
28579,28579,20211009_3064,2021-10-09,2007700,PEMA-HR1000,토,현진,동해,16.6,63.9,3.3,0.0,0.0,0.0,9.004791


In [31]:
# # 요일 변수 생성
# def weekday(x):
#     if x in ['월','화','수']: return 0
#     elif x in ['목','금','토','일']: return 1

# df['weekdayClass'] = df.weekday.apply(weekday)

In [32]:
# null 값 전처리

# Drop 하지 않는 숫자형 Null 칼럼들은 평균값으로 대체
df.fillna(df.mean(),inplace=True)

# Null 값이 있는 피처명과 타입 추출 -> 문자형은 널값 처리가 안되있음
null_column_count = df.isnull().sum()[df.isnull().sum()>0]
print('Null feature type:\n',df.dtypes[null_column_count.index])

Null feature type:
 Series([], dtype: object)


In [33]:
## 월별 기온 평균

df = df.astype({'solddate': 'str'})

q1_avg = df[df['solddate'].str.contains('-01-')]
q2_avg = df[df['solddate'].str.contains('-02-')]
q3_avg = df[df['solddate'].str.contains('-03-')]
q4_avg = df[df['solddate'].str.contains('-04-')]
q5_avg = df[df['solddate'].str.contains('-05-')]
q6_avg = df[df['solddate'].str.contains('-06-')]
q7_avg = df[df['solddate'].str.contains('-07-')]
q8_avg = df[df['solddate'].str.contains('-08-')]
q9_avg = df[df['solddate'].str.contains('-09-')]
q10_avg = df[df['solddate'].str.contains('-10-')]
q11_avg = df[df['solddate'].str.contains('-11-')]
q12_avg = df[df['solddate'].str.contains('-12-')]

q1m=q1_avg.d_day_2_tem.mean()
q2m=q2_avg.d_day_2_tem.mean()
q3m=q3_avg.d_day_2_tem.mean()
q4m=q4_avg.d_day_2_tem.mean()
q5m=q5_avg.d_day_2_tem.mean()
q6m=q6_avg.d_day_2_tem.mean()
q7m=q7_avg.d_day_2_tem.mean()
q8m=q8_avg.d_day_2_tem.mean()
q9m=q9_avg.d_day_2_tem.mean()
q10m=q10_avg.d_day_2_tem.mean()
q11m=q11_avg.d_day_2_tem.mean()
q12m=q12_avg.d_day_2_tem.mean()

In [34]:
# 월별 평균온도 (18~21년)
df['solddate'] = pd.to_datetime(df['solddate'],format= '%Y-%m-%d')
def m_temp(x):
    if x.month in [1]: return 0.952
    elif x.month in [2]: return 3.423
    elif x.month in [3]: return 8.197
    elif x.month in [4]: return 12.668
    elif x.month in [5]: return 18.327
    elif x.month in [6]: return 22.522
    elif x.month in [7]: return 24.52
    elif x.month in [8]: return 26.372
    elif x.month in [9]: return 21.134
    elif x.month in [10]: return 14.293
    elif x.month in [11]: return 7.403
    elif x.month in [12]: return 0.858
    
    

df['m_temp'] = df.solddate.apply(m_temp)

In [35]:
# 월별 강수 평균

df = df.astype({'solddate': 'str'})

q1_avg = df[df['solddate'].str.contains('-01-')]
q2_avg = df[df['solddate'].str.contains('-02-')]
q3_avg = df[df['solddate'].str.contains('-03-')]
q4_avg = df[df['solddate'].str.contains('-04-')]
q5_avg = df[df['solddate'].str.contains('-05-')]
q6_avg = df[df['solddate'].str.contains('-06-')]
q7_avg = df[df['solddate'].str.contains('-07-')]
q8_avg = df[df['solddate'].str.contains('-08-')]
q9_avg = df[df['solddate'].str.contains('-09-')]
q10_avg = df[df['solddate'].str.contains('-10-')]
q11_avg = df[df['solddate'].str.contains('-11-')]
q12_avg = df[df['solddate'].str.contains('-12-')]

q1r=q1_avg.d_day_2_tem_rainamount.mean()
q2r=q2_avg.d_day_2_tem_rainamount.mean()
q3r=q3_avg.d_day_2_tem_rainamount.mean()
q4r=q4_avg.d_day_2_tem_rainamount.mean()
q5r=q5_avg.d_day_2_tem_rainamount.mean()
q6r=q6_avg.d_day_2_tem_rainamount.mean()
q7r=q7_avg.d_day_2_tem_rainamount.mean()
q8r=q8_avg.d_day_2_tem_rainamount.mean()
q9r=q9_avg.d_day_2_tem_rainamount.mean()
q10r=q10_avg.d_day_2_tem_rainamount.mean()
q11r=q11_avg.d_day_2_tem_rainamount.mean()
q12r=q12_avg.d_day_2_tem_rainamount.mean()

In [36]:
# 월별 평균강수량 (18~21년)
df['solddate'] = pd.to_datetime(df['solddate'],format= '%Y-%m-%d')
def m_rain(x):
    if x.month in [1]: return 1.375
    elif x.month in [2]: return 2.182
    elif x.month in [3]: return 1.292
    elif x.month in [4]: return 2.629
    elif x.month in [5]: return 3.597
    elif x.month in [6]: return 4.404
    elif x.month in [7]: return 10.681
    elif x.month in [8]: return 9.717
    elif x.month in [9]: return 6.087
    elif x.month in [10]: return 0.708
    elif x.month in [11]: return 1.412
    elif x.month in [12]: return 0.616

df['m_rain'] = df.solddate.apply(m_rain)

In [37]:
# 판매 지역변수 -> 광역시,도별 단위로 범주화
def dododo(x):
    if x in ['성수','서울','송파','동서울', '이서', '풍납', '남부', '강서', '남서울','신림']: return '서울'
    
    elif x in ['수원','화성', '안양', '의왕', '용인', '영동', '동탄', '오산', '안산', '성남','천안', '평택', '천안','이천','여주','광주', '이천',\
               '안성', '옥산','동두천', '포천', '동두천', '양주', '의정부', '포천','양평','남양주', '구리', '하남', '진접', '덕소', '진영',\
               '파주', '고양', '일산','부천','김포','광명','미사리','비봉']: return '경기도'
    
    elif x in ['사천', '진주', '가좌','광양시','하동', '광양','산청','의령군', '함안', '의령', '창녕', '대산', '칠서',\
               '김해시','김해','함양군','함양','합천','마산', '진해', '창원','밀양', '창녕','양산시','양산','진주','사천',\
               '통영', '고성', '남포항','화도']: return '경상남도'
    
    elif x in ['의성','단촌','구미','약목', '옥계', '군위', '성주', '칠곡','영덕','울진','안동', '예천', '영양',\
               '영천','경산','포항','장흥','경주','문경','경산','고령']: return '경상북도'
    
    elif x in ['순창군' ,'담양', '순창','고창','장성','해남','여수','영광군',\
               '영광','목포', '무안', '해남','순천', '구례','장흥','나주','회주','완도','노화도','경광주','화순']: return '전라남도'
    
    elif x in ['익산', '군산', '군산한전주', '익산','전주','장수','전주', '완주','부안',\
               '남원','오수','임실','장성','정읍']: return '전라북도'
    
    elif x in ['서산', '당진', '태안','천안', '공주', '아산','금산','부여',\
               '청양', '논산', '계룡','홍성','천북', '보령','세종','연기','예산']: return '충청남도'
    
    elif x in ['청주', '청원', '옥천', '진천','제천','단양','성주','영동','충주', '음성','보은']: return '충청북도'
    
    elif x in ['홍성','북춘천', '춘천','원주','양양', '강릉','동해', '삼척','원주','횡성','속초','고성', '양양',\
               '정선군', '평창','영월','철원','현림수로']: return '강원도'
    
    elif x in ['대전광역시','대전','장동']: return '대전'
    
    elif x in ['대구','하빈']: return '대구'
    
    elif x in ['인천','송도','서인천']: return '인천'
    
    elif x in ['부산','회동동','서부산','신평','정관']: return '부산'
    
    elif x in ['울산','온산','언양',]: return '울산'
    
    elif x in ['세종']: return '세종'
    
    
    
df['DO'] = df.region.apply(dododo)

In [38]:
# 2021.04~2021.10년 데이터 병합용 인덱스와 판매일자 분리
df_index = df[['solddate','index']]

In [39]:
# 원핫 인코딩

print('ohe 전 shape:',df.shape)
df_ohe = pd.get_dummies(df[['index','m_rain','m_temp','prodname','weekday','log_ORDER_QUANT','log_SOLD_QUANT','DO']])
print('ohe 후 shape:',df_ohe.shape)

# Null값 확인
null_column_count = df_ohe.isnull().sum()[df_ohe.isnull().sum()>0]
print(df_ohe.dtypes[null_column_count.index])


ohe 전 shape: (28581, 20)
ohe 후 shape: (28581, 52)
Series([], dtype: object)


In [40]:
# index 기준으로 판매일자(solddate)병합
df_ohe1 = pd.merge(df_ohe,df_index)

df_ohe2 = df_ohe1.copy()


df_ohe1 = df_ohe1.astype({'solddate': 'str'})

In [41]:
# 원핫 인코딩후의 칼럼들의 타입과 Null값 확인
df_ohe1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28581 entries, 0 to 28580
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   28581 non-null  int64  
 1   m_rain                  28581 non-null  float64
 2   m_temp                  28581 non-null  float64
 3   log_ORDER_QUANT         28581 non-null  float64
 4   log_SOLD_QUANT          28581 non-null  float64
 5   prodname_AE             28581 non-null  uint8  
 6   prodname_CSA4000        28581 non-null  uint8  
 7   prodname_CSA4000(PCA)   28581 non-null  uint8  
 8   prodname_CSA5000        28581 non-null  uint8  
 9   prodname_PEMA-500FR     28581 non-null  uint8  
 10  prodname_PEMA-580FX     28581 non-null  uint8  
 11  prodname_PEMA-CR1000    28581 non-null  uint8  
 12  prodname_PEMA-CSA5000   28581 non-null  uint8  
 13  prodname_PEMA-HR1000    28581 non-null  uint8  
 14  prodname_PEMA-HR1000S   28581 non-null

In [42]:
# 데이터 병합을 위한 원핫 인코딩 후의 데이터 스플릿

# 2021년 4월~10월 데이터 추출
df1_ohe = df_ohe1[df_ohe1['solddate'].str.contains('2021-04-|2021-05-|\
2021-06-|2021-07-|2021-08-|2021-09-|2021-10-')]

# 2021년 4월~10월  데이터이외
df_ohe1 = df_ohe1[~df_ohe1.solddate.str.contains('2021-04-|2021-05-|\
2021-06-|2021-07-|2021-08-|2021-09-|2021-10-')]

In [43]:
df_ohe1.columns

Index(['index', 'm_rain', 'm_temp', 'log_ORDER_QUANT', 'log_SOLD_QUANT',
       'prodname_AE', 'prodname_CSA4000', 'prodname_CSA4000(PCA)',
       'prodname_CSA5000', 'prodname_PEMA-500FR', 'prodname_PEMA-580FX',
       'prodname_PEMA-CR1000', 'prodname_PEMA-CSA5000', 'prodname_PEMA-HR1000',
       'prodname_PEMA-HR1000S', 'prodname_PEMA-HR1500',
       'prodname_PEMA-PCM2000', 'prodname_PEMA-PCM2000B',
       'prodname_PEMA-PCR3000E', 'prodname_PEMA-PCR3000N',
       'prodname_PEMA-PR1000', 'prodname_PEMA-PR2000', 'prodname_PEMA-SN400',
       'prodname_PEMA-SP1000', 'prodname_PEMA-SPR', 'prodname_PEMA-SR2000',
       'prodname_PEMA-SR2000A', 'prodname_PEMA-SR3000F',
       'prodname_PEMA-SR5000F', 'prodname_PR1000', 'prodname_SRE-110',
       'prodname_SRE-200', 'weekday_금', 'weekday_목', 'weekday_수', 'weekday_월',
       'weekday_일', 'weekday_토', 'DO_강원도', 'DO_경기도', 'DO_경상남도', 'DO_경상북도',
       'DO_대구', 'DO_대전', 'DO_부산', 'DO_서울', 'DO_울산', 'DO_인천', 'DO_전라남도',
       'DO_전라북도', 'DO_충청남도

In [44]:
# 예측값 생성을 위한 데이터셋 스플릿

X_train = df_ohe1[['m_rain', 'm_temp','prodname_AE', 'prodname_CSA4000',\
                  'prodname_CSA4000(PCA)','prodname_CSA5000',\
                  'prodname_PEMA-500FR', 'prodname_PEMA-580FX',\
                  'prodname_PEMA-CR1000', 'prodname_PEMA-CSA5000', 'prodname_PEMA-HR1000',\
                  'prodname_PEMA-HR1000S', 'prodname_PEMA-HR1500',\
                  'prodname_PEMA-PCM2000', 'prodname_PEMA-PCM2000B',\
                  'prodname_PEMA-PCR3000E', 'prodname_PEMA-PCR3000N',\
                  'prodname_PEMA-PR1000', 'prodname_PEMA-PR2000', 'prodname_PEMA-SN400',\
                  'prodname_PEMA-SP1000', 'prodname_PEMA-SPR', 'prodname_PEMA-SR2000',\
                  'prodname_PEMA-SR2000A', 'prodname_PEMA-SR3000F',\
                  'prodname_PEMA-SR5000F', 'prodname_PR1000', 'prodname_SRE-110',\
                  'prodname_SRE-200', 'weekday_금', 'weekday_목', 'weekday_수', 'weekday_월',\
                  'weekday_일', 'weekday_토', 'DO_강원도', 'DO_경기도', 'DO_경상남도', 'DO_경상북도',\
                  'DO_대구', 'DO_대전', 'DO_부산', 'DO_서울', 'DO_울산', 'DO_인천', 'DO_전라남도',\
                  'DO_전라북도', 'DO_충청남도', 'DO_충청북도','log_ORDER_QUANT']]

y_train = df_ohe1[['log_SOLD_QUANT']]

X_test = df1_ohe[['m_rain', 'm_temp','prodname_AE', 'prodname_CSA4000',\
                  'prodname_CSA4000(PCA)','prodname_CSA5000',\
                  'prodname_PEMA-500FR', 'prodname_PEMA-580FX',\
                  'prodname_PEMA-CR1000', 'prodname_PEMA-CSA5000', 'prodname_PEMA-HR1000',\
                  'prodname_PEMA-HR1000S', 'prodname_PEMA-HR1500',\
                  'prodname_PEMA-PCM2000', 'prodname_PEMA-PCM2000B',\
                  'prodname_PEMA-PCR3000E', 'prodname_PEMA-PCR3000N',\
                  'prodname_PEMA-PR1000', 'prodname_PEMA-PR2000', 'prodname_PEMA-SN400',\
                  'prodname_PEMA-SP1000', 'prodname_PEMA-SPR', 'prodname_PEMA-SR2000',\
                  'prodname_PEMA-SR2000A', 'prodname_PEMA-SR3000F',\
                  'prodname_PEMA-SR5000F', 'prodname_PR1000', 'prodname_SRE-110',\
                  'prodname_SRE-200', 'weekday_금', 'weekday_목', 'weekday_수', 'weekday_월',\
                  'weekday_일', 'weekday_토', 'DO_강원도', 'DO_경기도', 'DO_경상남도', 'DO_경상북도',\
                  'DO_대구', 'DO_대전', 'DO_부산', 'DO_서울', 'DO_울산', 'DO_인천', 'DO_전라남도',\
                  'DO_전라북도', 'DO_충청남도', 'DO_충청북도','log_ORDER_QUANT']]

y_test = df1_ohe[['log_SOLD_QUANT']]

In [45]:
# 검증한 모델(XGBRegressor)을 통한 예측값 생성

from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error,r2_score

# XGBRegressor 로 예측값 생성

xgb_reg = XGBRegressor(n_estimators=1000, learning_rate=0.05, 
                       colsample_bytree=0.5, subsample=0.8)


# xgb모델의 학습
xgb_reg.fit(X_train, y_train)


# xgb 모델 예측
xgb_pred = xgb_reg.predict(X_test)


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.8,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [46]:
# 생성된 예측값 병합
df1 = pd.DataFrame(X_test)
df2 = pd.DataFrame(xgb_pred)
df1.reset_index(inplace=True)
df_predicted =pd.concat([df1,df2],axis=1)
df_predicted = df_predicted.rename({'log_SOLD_QUANT':'True_values',0:'Predicted_values'},axis='columns')


In [47]:
# 로그화된 실제수주량 원복
df_predicted['order_quant'] = np.expm1(df_predicted.log_ORDER_QUANT)
df_predicted['sold_quant'] = np.expm1(df_predicted.Predicted_values)

In [48]:
# 데이터 프레임 원본데이터화 
df_merge = pd.merge(df_predicted,df_pred_21_03,how='outer',on='index')

# 데이터 프레임에서 원본데이터 칼럼만 선택 후 예측값 포함 DB용 데이터셋생성
df_csv = df_merge[['orderid','solddate','custid',\
                  'prodname','weekday','sitebusname','region',\
                  'd_day_2_tem','d_day_2_hum','d_day_2_wind',\
                  'd_day_2_tem_rainamount','d_day_2_rainhour',\
                  'order_quant','sold_quant']]
df_csv


,orderid,solddate,custid,prodname,weekday,sitebusname,region,d_day_2_tem,d_day_2_hum,d_day_2_wind,d_day_2_tem_rainamount,d_day_2_rainhour,order_quant,sold_quant
0,AE21032954,2021-04-01,2006200,AE,목,신일,송파,14.9,80.8,3.1,56.2,16.5,1656.0,1802.625000
1,AE21032953,2021-04-01,2006100,AE,목,삼표,용인,15.4,85.4,1.9,46.7,14.6,686.0,916.102661
2,AE21032952,2021-04-01,2006000,AE,목,삼표,풍납,14.9,80.8,3.1,56.2,16.5,989.0,1035.230225
3,AE21032951,2021-04-01,2005900,AE,목,삼표,성수,14.9,80.8,3.1,56.2,16.5,2626.0,2461.819336
4,HR1021032950,2021-04-01,2001600,PEMA-HR1000,목,성진,인천,14.1,79.6,3.1,54.7,16.5,8280.0,9310.599609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5085,20211009_949,2021-10-09,2007400,CSA5000,토,승일,함안,18.6,55.4,3.4,0.0,0.0,6396.0,8251.208984
5086,20211009_3062,2021-10-09,2007500,PEMA-HR1000,토,삼우,경산,17.8,59.1,3.7,0.0,0.0,7602.0,7978.093262
5087,20211009_3063,2021-10-09,2007600,PEMA-HR1000,토,한빛,부안,17.2,64.6,1.7,0.0,0.0,5836.0,8308.867188
5088,20211009_3064,2021-10-09,2007700,PEMA-HR1000,토,현진,동해,16.6,63.9,3.3,0.0,0.0,8141.0,8174.661133


### 위에서 예측된 생산량 값을 pred_sales 테이블에 저장

In [ ]:
conn = pymysql.connect(host='127.0.0.1',
                      user='root',
                      password='maria',
                      database='aps',
                      port=3306,
                      charset='utf8',
                      autocommit=True,
                      cursorclass=pymysql.cursors.DictCursor
                      )
cursor = conn.cursor()

sql_ins = """INSERT INTO pred_sales(orderid, solddate, custid, prodname, weekday, sitebusname, region, \
                                    d_day_2_tem, d_day_2_hum, d_day_2_wind, d_day_2_tem_rainamount, d_day_2_rainhour, \
                                    order_quant, sold_quant) \
             VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) """

for i in range(0, len(df_csv)):
    cursor.execute(sql_ins, (df_csv['orderid'][i], df_csv['solddate'][i], df_csv['custid'][i], df_csv['prodname'][i], 
                             df_csv['weekday'][i], df_csv['sitebusname'][i], df_csv['region'][i], 
                             df_csv['d_day_2_tem'][i], df_csv['d_day_2_hum'][i], df_csv['d_day_2_wind'][i], 
                             df_csv['d_day_2_tem_rainamount'][i], df_csv['d_day_2_rainhour'][i],
                             df_csv['order_quant'][i], df_csv['sold_quant'][i])) 
    conn.commit()
    
conn.close()


### 생산량 예측 결과가 저장된 DB 정보 확인

In [ ]:
conn = pymysql.connect(host='127.0.0.1',
                      user='root',
                      password='maria',
                      database='aps',
                      port=3306,
                      charset='utf8',
                      autocommit=True,
                      cursorclass=pymysql.cursors.DictCursor
                      )
cursor = conn.cursor()

sql = " SELECT * FROM pred_sales "

cursor.execute(sql)
row = cursor.fetchall() 
conn.close()

In [ ]:
# 2021 4~10월 수주예측 데이터셋 불러오기
df_2021_4_10 = pd.DataFrame(row)
df_2021_4_10

In [ ]:
# 2021.04~2021.10월 병합 데이터 Null값 확인
df_2021_4_10.isnull().sum()

In [ ]:
df_2021_4_10

In [ ]:
df_pred_21_03x

In [ ]:
# 2018~2021.10 까지의 전체데이터

df_total_csv = pd.concat([df_2021_4_10,df_pred_21_03x])

In [ ]:
# 2018~2021.10 까지의 전체데이터
df_total_csv = df_total_csv[['orderid','solddate','custid',\
                             'prodname','weekday','sitebusname','region',\
                             'd_day_2_tem','d_day_2_hum','d_day_2_wind',\
                             'd_day_2_tem_rainamount','d_day_2_rainhour',\
                             'order_quant','sold_quant']]

df_total_csv

In [ ]:
# 2018.01~2021.10까지의 수주예측 데이터셋 생성 및 불러오기
df_total_csv.to_csv('Project3_data/2021_total_판매실행계획.csv',encoding='euckr')
df_total_csv = pd.read_csv('Project3_data/2021_total_판매실행계획.csv',encoding='euckr')
df_total_csv

In [ ]:
# 2018.01~2021.10 까지 병합데이터 Null값 확인
df_total_csv.isnull().sum()

In [ ]:
# 실제 수주량 총합과 주문 수주량 총합과의 차이
df04_ssum = df_2021_4_10['sold_quant'].astype('float').sum()
df04_osum = df_2021_4_10['order_quant'].astype('float').sum()

print("[2021년 4~10월 사전 주문 수주량의 총 합은] :",'(',df04_osum,')',"입니다",'\n')
print("[2021년 4~10월 실제 판매 수주량의 총 합은] :",'(',np.round(df04_ssum,3),')',"입니다",'\n')
print("[2021년 4~10월 전체 사전 주문 수주량과 전체 실제 판매 수주량의 차이의 비는] :",'(',(np.round(df04_osum/df04_ssum,3)),')',"입니다",'\n')


### 원자재 소요 계획 생성 - 예측된 생산량에 제품별 원자재 투입 비율을 적용해서 DB에 저장
- 운영에 적용 시에는 제품별 원자재 투입 비율 정보가 별도로 존재하는지 확인 필요

In [13]:
conn = pymysql.connect(host='127.0.0.1',
                      user='root',
                      password='maria',
                      database='aps',
                      port=3306,
                      charset='utf8',
                      autocommit=True,
                      cursorclass=pymysql.cursors.DictCursor
                      )
cursor = conn.cursor()

sql_ins_sel = """ INSERT INTO pred_purchase (sold_date, material_code, material_name, order_quant) 
                  SELECT solddate, material_code, material_name, SUM(input_quant) AS order_quant 
                  FROM (     
                        SELECT b.solddate, a.prodname, a.material_code, a.material_name, a.input_rate, b.sold_quant, 
                               ROUND(b.sold_quant*(a.input_rate/100)) AS input_quant 
                        FROM m_input_rate a,  
                             (SELECT solddate, prodname, SUM(sold_quant) sold_quant 
                              FROM pred_sales  
                              WHERE solddate BETWEEN '2021-04-01' AND '2021-10-31' 
                              GROUP BY solddate, prodname) b 
                        WHERE a.prodname = b.prodname ) t 
                  GROUP BY solddate, material_code, material_name """
cursor.execute(sql_ins_sel) 
conn.commit()
conn.close()

2499

#### 위에서 추출한 일별 원자재 소요 정보를 csv 파일로 저장.

In [3]:
conn = pymysql.connect(host='127.0.0.1',
                      user='root',
                      password='maria',
                      database='aps',
                      port=3306,
                      charset='utf8',
                      autocommit=True,
                      cursorclass=pymysql.cursors.DictCursor
                      )
cursor = conn.cursor()

sql_sel = """ SELECT solddate, prodname, material_code, material_name, SUM(input_quant) AS order_quant 
              FROM (     
                    SELECT b.solddate, a.prodname, a.material_code, a.material_name, a.input_rate, b.sold_quant, 
                           ROUND(b.sold_quant*(a.input_rate/100)) AS input_quant 
                    FROM m_input_rate a,  
                         (SELECT solddate, prodname, SUM(sold_quant) sold_quant 
                          FROM pred_sales  
                          WHERE solddate BETWEEN '2021-04-01' AND '2021-10-31' 
                          GROUP BY solddate, prodname) b 
                    WHERE a.prodname = b.prodname ) t 
              GROUP BY solddate, prodname, material_code, material_name """
cursor.execute(sql_sel) 
row = cursor.fetchall() 
conn.close()

df_mat = pd.DataFrame(row)
df_mat.to_csv('material_requirement.csv', encoding='euckr')

9585